## QA 简介  

问答系统与目前主流检索技术有两点不同：首先是查询方式为完整而口语化的问句，其回传的为高精准度网页结果或明确的答案字串。面对这种系统，使用者不需要费心去一一检视搜索引擎回传的网页，对于资讯检索的效率与资讯的普及都有很大帮助。从系统内部来看，问答系统使用了大量有别于传统资讯检索系统自然语言处理技术，如自然语言解析（Natural Language Parsing）、问题分类（Question Classification）、实体识别（Named Entity Recognition）等等。少数系统甚至会使用复杂的逻辑推理机制，来区分出需要推理机制才能够区隔出来的答案。  

## DrQA  


### 框架简介

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/QA/Drqa/1.jpg?raw=true)
<center> **DrQA 流程** </center >

DrQA 是一个应用于开放域问答的阅读理解系统。目标是解决 "machine reading at scale"（MRS）问题。从非常大的非结构化文档集中搜索特定问题的答案（文档集可能是冗余的）。因此，系统需要共同解决文档检索（查找相关文档）与机器阅读理解理解（从这些文档中识别答案）的问题。

DrQA 侧重于回答 factoid 问题，使用 Wikipedia 作为问题的知识源。为了回答任何问题，必须首先检索超过 500 万个中的少数可能相关的文章，然后仔细扫描以确定答案。

DrQA 包含两个模块，1、Document Retriever：用于查找和检索相关的文档；2、Document Reader：从检索到的文档中提取问题的答案。


### 介绍  

DrQA 使用维基百科作为独特知识源回答开放域中的 factoid 问题。维基百科是一个不断发展的详细信息来源，如果能够充分利用的话可以用来构建智能系统。与 Freebase 或 DBPedia 等知识库（KB）不同，这些知识库更容易处理，但不适合用于开放域问题解答，维基百科包含人类感兴趣的最新知识。然而，它是为人类而非机器设计的。  

使用维基百科文章作为知识源的问答（QA）的任务需面对大规模开放域 QA 和文本阅读理解两个难题。为了回答任何问题，首先必须检索超过 500 万个文章中的少数相关文章，然后仔细扫描以确定答案。将维基百科视为文章集合，不依赖于其内部图结构。因此，方法是通用的，可用于其他文档，书籍甚至每日更新的报纸。  

像 IBM 的 DeepQA 这样的大规模 QA 系统依赖于多个源来回答问题：除了维基百科之外，还使用了 KBs，词典，甚至新闻文章，书籍等，信息源之间的信息冗余会对结果造成一定影响。拥有单一知识源会强制模型在搜索答案时非常精确，因为答案可能只出现一次。因此，能够需要尽可能的提高机器的阅读能力，目前已经有很多用于评测机器阅读理解能力的数据集，如 SQuAD，CNN/Daily Mail 和 CBT等。  

然而，这些机器阅读理解数据通常假设已经识别出一小段相关文本并将其提供给模型，这对于构建 opendomain QA 系统是不现实的。与此形成鲜明对比的是，使用 KBs 或文档信息检索的方法必须将搜索作为解决方案的一个组成部分。  

文本也展示了如何使用多个现有的 QA 数据集来评估 MRS。DrQA 系统包括：（1）Document Retriever，使用 bigram hashing 和 TF-IDF 匹配的方法，检索出给定问题的相关文档集；（2）Document Reader，一个多层递归神经网络机器阅读理解模型，用于检测那些返回文档中的答案。  

实验表明，Document Retriever 优于内置的维基百科搜索引擎，而 Document Reader 在竞争激烈的 SQuAD 基准测试中达到了最先进的结果。最后，使用多个基准评估完整的系统。特别的，与单任务训练相比，通过使用多任务学习和远程监督，所有数据集的性能都得到了提高。  

### DrQA 框架介绍  

#### Document Retriever  

遵循经典的 QA 系统，首先使用高效（非机器学习）文档检索系统来缩小搜索空间，并专注于阅读可能相关的文章。与内置的基于 ElasticSearch 的Wikipedia Search API 相比，简单的倒排索引查之后使用 term vector model scoring 在许多问题类型上表现的都相当好。使用 TF-IDF 加权的 bag-of-word 向量来比较问题和文章的相关度。通过与 n-gram 特征相结合来进一步改进检索系统。使用 unsigned murmur3 hash 方法将 bigrams 映射到 $2^{24}$ 个 bins 的方法可以在保证检索速度和内存的情况下达到最好的效果。  

对每个问题都返回最相关的 5 篇文章，用于之后的 Document Reader。  


**Document Retriever** 步骤：  

- 1、构建 bigrams hash 矩阵，求每篇 wiki 文档的bigrams，hash 每个 bigram，计算 tfidf值，hash 矩阵的维度为：hash_bins * num_docs  

- 2、计算 question 文本的 bigrams 并 hash，计算 hash 值的 tfidf，question 向量维度为：1 * hash_bins  

- 3、计算 question 向量与文档 hash 矩阵的矩阵乘积，乘积结果维度为： 1 * num_docs，计算出每篇 wiki 文档与 question 的相似度，相似度最大的 5 篇作为候选文档返回


#### Document Reader  

Document Reader 受到了当前阅读理解任务的启发，给出一个包含 $l$ 个 tokens 的问题 $q,{q1 ,...，ql}$ 和包含 $n$ 个段落的文档或文档集合，其中单个段落 $p$ 由 $m$ 个 tokens ${p1 ,...，pm}$ 组成，使用一个 RNN 模型，依次应用于每个段落，然后最终聚合预测的答案。  

**Paragraph encoding**  将段落中的每个词转为向量，然后使用多层双向 lstm 将这些词进行编码：  

${p1, . . . , pm} = RNN({\hat p1, . . . , \hat pm})$  

其中 $pi$ 是将对应单词编码为含有上下文新的向量，是每层 lstm 编码之后 concat 的结果。  

$\hat pi$ 向量是由以下特征组成的：

- Word embeddings: 使用 840B Web crawl data 训练的 300-dimensional Glove word embeddings，训练阅读理解模型的时候固定了大部分的词向量，只 fine-tune 了 1000 个最常见单词对应的词向量。  

- Exact match: 使用三个简单的二值特征来表示单词 $pi$ 是否是问题中的某个单词，可以使原始单词，小写或简称，这些特征对最终结果很有帮助。  

- Token features: 还使用了一些手工特征，$f_{token}(pi) = (POS(pi), NER(pi), TF(pi))$, TF-term frequency  

- Aligned question embedding: 使用 question 特征与段落计算 attention 得到的组合特征  

**Question encodin** question encodin 比较简单，只在 question  embedding 上使用另外一个 RNN 进行编码：$ {q1,..., ql} → q$, 并计算每个单词的重要性，将编码之后的向量进行加权求和 $q = \sum_j b_jqj$。  

**Predictio** 在段落级别，目标是预测最可能是正确答案的 span of token。将段落向量和问题向量作为输入，并简单地单独训练两个分类器以预测 span 的两端。  

### Data  

论文主要使用了三种数据：（1）作为寻找答案知识源的 Wikipedia，（2）SQuAD 数据集，是训练 Document Reader 的主要资源;（3）除了 SQuAD 之外的三个 QA 数据集（CuratedTREC, WebQuestions and WikiMovies），用于测试整个系统开放域的 QA 能力，并评估模型从多任务学习和远程监督中学习的能力。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/QA/Drqa/2.jpg?raw=true)
<center> **数据简介** </center >

#### Wikipedia  

使用 2016-12-21 dump 的英语维基百科作为所有实验获取答案的知识源。从每个页面中仅提取纯文本，并删除所有结构化数据部分（如列表和图形）。保留了 5,075,182 篇文章，其中包含 9,008,962 去掉大小写和重音标志的 tokens。  

####  SQuAD  

Stanford Question Answering Dataset（SQuAD）是一个基于维基百科的机器理解数据集。该数据集包含 87k 个用于训练的示例和用于 dev 的 10k 个示例，以及一个只能由 SQuAD 创建者访问的大型隐藏测试集。每个示例由从维基百科文章中提取的段落和相关的人工生成的问题组成。答案是这一段的一个 span。使用两个评估指标：精确字符串匹配（EM）和 F1 分数。  

#### Open-domain QA Evaluation Resources  

SQuAD 是目前可用的最大通用 QA 数据集之一。SQUAD 问题是通过向每个注释者显示一个段落并要求他们写一个问题收集的，它们的分布非常具体。因此，建议在以不同的方式构建的开放域 QA 数据集上训练和评估我们的系统。  

**CuratedTREC**  该数据集是 TREC QA 任务的数据集，包含从 TREC 1999,2000,2001 和 2002 的数据集中提取的总共 2,180 个问题。  

**WebQuestions**  该数据集用于回答 Freebase KB 中的问题。通过 Google Suggest API 抓取问题，然后使用 Amazon Mechanical Turk 获取答案而创建的。  

**WikiMovies**  该数据集包含在电影领域的 96k 问答对。  

#### Distantly Supervised Data  

上面提供的所有 QA 数据集都包含训练集，但 CuratedTREC，WebQuestions 和 WikiMovies 只包含问答配对，而不是类似 SQuAD 中的相关文档或段落，因此不能直接用于训练 Document Reader。隐私使用远程监督的方法，使用一个程序自动将段落与这些训练样本相关联，然后将这些数据添加到训练集中。  

对每个question answer 对使用以下过程来构建训练集。首先，在问题上运行 Document Retriever 以检索前 5 篇维基百科文章。这些文章中没有与已知答案完全匹配的所有段落都将被直接丢弃。所有短于 25 或超过 1500 个字符的段落也会被过滤掉。如果在问题中检测到实体名字，删除任何不包含它们的段落。对于每个检索到的页面中的剩余段落，使用 unigram 和 bigram 在问题和 20 个 token 窗口之间重叠来对所有匹配答案的位置进行评分，并保持最重叠的前 5 个段落。  

请注意，还可以通过尝试查找不仅在提供的段落中的答案，还从其他页面或给定段落所在的同一页面中查找答案的提示来生成 SQuAD 的 DS 数据。观察到大约一半的 DS 示例来自 SQuAD 中使用的文章之外的页面。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/QA/Drqa/3.jpg?raw=true)

### Experiments  

#### Finding Relevant Articles  

首先检查 Document Retriever 模块在所有 QA 数据集上的性能。表3 比较了用于查找包含问题答案文章的两种方法与维基百科搜索引擎方法的性能。具体来说。所有数据集的结果表明我们的简单方法优于维基百科搜索，尤其是使用 bigram 哈希。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/QA/Drqa/4.jpg?raw=true)
<center> **Document Retriever 结果** </center >

#### Reader Evaluation on SQuAD  

在标准 SQuAD 评估中评估 Document Reader  

**Implementation details**  使用 3 层双向 LSTM，其中 h = 128 个隐藏单元用于段落和问题编码。使用 Stanford CoreNLP 工具包进行分词，并生成 lemma，词性和命名实体识别。  

**Result and analysis**  表4 列出了在开发和测试集上的评估结果。自创建以来，SQuAD 一直是一个非常具有竞争力的机器理解基准，表中只列出了性能最佳的系统。我们的系统（单一模型）在测试集上可以达到 70.0% 的精确匹配和 79.0% 的 F1 分数，超过了所有已发布的结果。  

#### Full Wikipedia Question Answering  

最后，使用上节中介绍的四个数据集评估 DrQA 完整的系统。比较了三个版本的 DrQA，评估了在提供给 Document Reader 的训练资源中使用远程监督和多任务学习的影响：  

- SQuAD：仅在 SQuAD 训练集上训练的单个 Document Reader 模型。  

- 微调（DS）：Document Reader 模型在 SQuAD 上预先训练，然后使用其远程监督（DS）训练集独立地对每个数据集进行微调。  

- 多任务（DS）：在 SQuAD 训练集和所有 DS 数据集上联合训练的单个 Document Reader 模型。  

**Results**  表6列出了结果。无约束QA与机器理解（给出正确的段落）和相比，更加困难，但 DrQA 仍然在所有四个数据集中提供较好的性能。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/QA/Drqa/5.jpg?raw=true)
<center> **Document Reader 结果** </center >  

## 参考  

1、[Reading Wikipedia to Answer Open-Domain Questions](https://arxiv.org/pdf/1704.00051.pdf)  